### Fall 2024: CIS 556- Database Systems

### Topic - Library inventory optimization using the Goodreads Books dataset
Team Members - Sagnik Roy, Shubhi Gupta


In [1]:
import pandas as pd
import re

# Load your dataset
books_data = pd.read_csv("data/books.csv", engine="python", on_bad_lines="skip", usecols=lambda col: col != "Unnamed: 12")

# Display the updated dataset's structure and the first few rows
books_data.info(), books_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11123 entries, 0 to 11122
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   bookID              11123 non-null  int64 
 1   title               11123 non-null  object
 2   authors             11123 non-null  object
 3   average_rating      11123 non-null  object
 4   isbn                11123 non-null  object
 5   isbn13              11123 non-null  object
 6   language_code       11123 non-null  object
 7     num_pages         11123 non-null  object
 8   ratings_count       11123 non-null  int64 
 9   text_reviews_count  11123 non-null  int64 
 10  publication_date    11123 non-null  object
 11  publisher           11123 non-null  object
dtypes: int64(3), object(9)
memory usage: 1.0+ MB


(None,
    bookID                                              title  \
 0       1  Harry Potter and the Half-Blood Prince (Harry ...   
 1       2  Harry Potter and the Order of the Phoenix (Har...   
 2       4  Harry Potter and the Chamber of Secrets (Harry...   
 3       5  Harry Potter and the Prisoner of Azkaban (Harr...   
 4       8  Harry Potter Boxed Set  Books 1-5 (Harry Potte...   
 
                       authors average_rating        isbn         isbn13  \
 0  J.K. Rowling/Mary GrandPré           4.57  0439785960  9780439785969   
 1  J.K. Rowling/Mary GrandPré           4.49  0439358078  9780439358071   
 2                J.K. Rowling           4.42  0439554896  9780439554893   
 3  J.K. Rowling/Mary GrandPré           4.56  043965548X  9780439655484   
 4  J.K. Rowling/Mary GrandPré           4.78  0439682584  9780439682589   
 
   language_code   num_pages  ratings_count  text_reviews_count  \
 0           eng         652        2095690               27591   
 1       

In [2]:
books_data.columns

Index(['bookID', 'title', 'authors', 'average_rating', 'isbn', 'isbn13',
       'language_code', '  num_pages', 'ratings_count', 'text_reviews_count',
       'publication_date', 'publisher'],
      dtype='object')

In [3]:
# Ensure all titles are strings
books_data['title'] = books_data['title'].astype(str)

# Step 2: Display the updated dataset to check
print(books_data[['title']].head())


                                               title
0  Harry Potter and the Half-Blood Prince (Harry ...
1  Harry Potter and the Order of the Phoenix (Har...
2  Harry Potter and the Chamber of Secrets (Harry...
3  Harry Potter and the Prisoner of Azkaban (Harr...
4  Harry Potter Boxed Set  Books 1-5 (Harry Potte...


In [4]:
books_data.shape

(11123, 12)

In [5]:
# Step 1: Define a function to clean the author names
def clean_author_names(author):
    if pd.isna(author):
        return None
    # Remove leading non-alphabetic characters and trim extra spaces
    author = re.sub(r'^[^a-zA-Z]+', '', author).strip()
    # Clean up any non-alphabetic characters, leaving only letters, spaces, or slashes
    author = re.sub(r'[^a-zA-Z\s/]', '', author)
    # Split authors by '/' if there are multiple authors
    authors = author.split('/')
    # Clean each author's name: strip extra spaces and remove any empty string
    authors = [name.strip() for name in authors if name.strip()]
    return '/'.join(authors) if authors else None

# Step 2: Apply the cleaning function to the 'author' column
books_data['authors'] = books_data['authors'].apply(clean_author_names)

# Display the cleaned dataset's structure and the first few rows
books_data.info(), books_data.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11123 entries, 0 to 11122
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   bookID              11123 non-null  int64 
 1   title               11123 non-null  object
 2   authors             11123 non-null  object
 3   average_rating      11123 non-null  object
 4   isbn                11123 non-null  object
 5   isbn13              11123 non-null  object
 6   language_code       11123 non-null  object
 7     num_pages         11123 non-null  object
 8   ratings_count       11123 non-null  int64 
 9   text_reviews_count  11123 non-null  int64 
 10  publication_date    11123 non-null  object
 11  publisher           11123 non-null  object
dtypes: int64(3), object(9)
memory usage: 1.0+ MB


(None,
    bookID                                              title  \
 0       1  Harry Potter and the Half-Blood Prince (Harry ...   
 1       2  Harry Potter and the Order of the Phoenix (Har...   
 2       4  Harry Potter and the Chamber of Secrets (Harry...   
 3       5  Harry Potter and the Prisoner of Azkaban (Harr...   
 4       8  Harry Potter Boxed Set  Books 1-5 (Harry Potte...   
 
                    authors average_rating        isbn         isbn13  \
 0  JK Rowling/Mary GrandPr           4.57  0439785960  9780439785969   
 1  JK Rowling/Mary GrandPr           4.49  0439358078  9780439358071   
 2               JK Rowling           4.42  0439554896  9780439554893   
 3  JK Rowling/Mary GrandPr           4.56  043965548X  9780439655484   
 4  JK Rowling/Mary GrandPr           4.78  0439682584  9780439682589   
 
   language_code   num_pages  ratings_count  text_reviews_count  \
 0           eng         652        2095690               27591   
 1           eng         87

In [6]:
import re

# Define a function to clean and standardize ISBN numbers
def clean_isbn(isbn):
    # Ensure the input is a string and handle any NaN values
    isbn = str(isbn).strip()
    
    # Remove unwanted characters but keep 'X' if it appears at the end
    isbn_cleaned = re.sub(r'[^0-9X]', '', isbn)
    
    # Return cleaned ISBN as text
    return isbn_cleaned

# Step 1: Clean the 'isbn' column using the function defined above
books_data['isbn'] = books_data['isbn'].apply(clean_isbn)

# Step 2: Clean the 'isbn13' column using the same function
books_data['isbn13'] = books_data['isbn13'].apply(clean_isbn)

# Check the cleaned ISBN and ISBN13 columns
print(books_data[['isbn', 'isbn13']].head())



         isbn         isbn13
0  0439785960  9780439785969
1  0439358078  9780439358071
2  0439554896  9780439554893
3  043965548X  9780439655484
4  0439682584  9780439682589


In [7]:
books_data.shape

(11123, 12)

In [8]:
# Convert 'average_rating' to numeric, invalid parsing will be set as NaN
books_data['average_rating'] = pd.to_numeric(books_data['average_rating'], errors='coerce')

# Count how many rows will be removed (those where 'average_rating' is NaN)
invalid_ratings = books_data[books_data['average_rating'].isna()]
count_invalid_ratings = len(invalid_ratings)

# Remove rows where 'average_rating' is NaN (invalid numbers)
books_data = books_data.dropna(subset=['average_rating'])

In [9]:
# Define a function to remove digits from language codes
def remove_numbers_from_language_code(language_code):
    # Use regex to remove digits
    return re.sub(r'\d+', '', str(language_code))

# Apply the function to the 'language_code' column
books_data['language_code'] = books_data['language_code'].apply(remove_numbers_from_language_code)

# Define a mapping of language codes to their full names
language_mapping = {
    'eng': 'English',
    'en-US': 'English (US)',
    'fre': 'French',
    'spa': 'Spanish',
    'en-GB': 'English (UK)',
    'mul': 'Multiple Languages',
    'grc': 'Ancient Greek',
    'enm': 'Middle English',
    'en-CA': 'English (Canada)',
    'ger': 'German',
    'jpn': 'Japanese',
    'ara': 'Arabic',
    'nl': 'Dutch',
    'zho': 'Chinese',
    'lat': 'Latin',
    'por': 'Portuguese',
    'srp': 'Serbian',
    'ita': 'Italian',
    'rus': 'Russian',
    'msa': 'Malay',
    'glg': 'Galician',
    'wel': 'Welsh',
    'swe': 'Swedish',
    'nor': 'Norwegian',
    'tur': 'Turkish',
    'gla': 'Gaelic',
    'ale': 'Aleut'
}

# Create a new column 'language_name' by applying the mapping
books_data['language_name'] = books_data['language_code'].map(language_mapping)

# Drop the old 'language_code' column
books_data.drop(columns=['language_code'], inplace=True)




In [10]:
books_data.shape

(11119, 12)

In [11]:
# Step 1: Remove rows where the 'publisher' column is null
books_data = books_data.dropna(subset=['publisher'])

# Step 2: Optionally display the updated dataset
print(books_data[['publisher']].head())

# Step 4: Drop rows where the 'publication_date' column is NaT (Not a Time)
books_data = books_data.dropna(subset=['publication_date'])

# Step 5: Convert 'publication_date' to datetime with correct format handling
# Adjust the format to handle different date formats like "yy/mm/dd"
books_data['publication_date'] = pd.to_datetime(books_data['publication_date'], errors='coerce', dayfirst=True)

# Step 6: Ensure that the 'publication_date' is in the correct format for PostgreSQL (yyyy-mm-dd)
books_data['publication_date'] = books_data['publication_date'].dt.strftime('%Y-%m-%d')

# Step 7: Extract unique publisher names from the books_data
publishers = set(books_data['publisher'].dropna())  # Remove NaN values before extracting unique publishers

print(books_data.head())


         publisher
0  Scholastic Inc.
1  Scholastic Inc.
2       Scholastic
3  Scholastic Inc.
4       Scholastic
   bookID                                              title  \
0       1  Harry Potter and the Half-Blood Prince (Harry ...   
1       2  Harry Potter and the Order of the Phoenix (Har...   
2       4  Harry Potter and the Chamber of Secrets (Harry...   
3       5  Harry Potter and the Prisoner of Azkaban (Harr...   
4       8  Harry Potter Boxed Set  Books 1-5 (Harry Potte...   

                   authors  average_rating        isbn         isbn13  \
0  JK Rowling/Mary GrandPr            4.57  0439785960  9780439785969   
1  JK Rowling/Mary GrandPr            4.49  0439358078  9780439358071   
2               JK Rowling            4.42  0439554896  9780439554893   
3  JK Rowling/Mary GrandPr            4.56  043965548X  9780439655484   
4  JK Rowling/Mary GrandPr            4.78  0439682584  9780439682589   

    num_pages  ratings_count  text_reviews_count publication_d

/var/folders/hr/t8t52lgs72v6zwxb59z155mm0000gn/T/ipykernel_33439/3096432006.py:12: UserWarning: Parsing dates in %m/%d/%Y format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  books_data['publication_date'] = pd.to_datetime(books_data['publication_date'], errors='coerce', dayfirst=True)


In [12]:
books_data.shape

(11119, 12)

In [13]:
# Step 8: Create a DataFrame for publishers with unique IDs
publishers_df = pd.DataFrame(publishers, columns=['name'])
publishers_df['publisherID'] = range(1, len(publishers_df) + 1)

# Step 9: Merge the 'publishers_df' DataFrame with 'books_data' to add 'publisherID'
books_data = books_data.merge(publishers_df, left_on='publisher', right_on='name', how='left')

# Step 10: Remove the 'name' column as it's now redundant
books_data = books_data.drop(columns=['name'])

# Step 11: Remove rows where 'publisher' or 'publication_date' is NULL
books_data = books_data.dropna(subset=['publisher', 'publication_date'])

books_data.info()


<class 'pandas.core.frame.DataFrame'>
Index: 11117 entries, 0 to 11118
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   bookID              11117 non-null  int64  
 1   title               11117 non-null  object 
 2   authors             11117 non-null  object 
 3   average_rating      11117 non-null  float64
 4   isbn                11117 non-null  object 
 5   isbn13              11117 non-null  object 
 6     num_pages         11117 non-null  object 
 7   ratings_count       11117 non-null  int64  
 8   text_reviews_count  11117 non-null  int64  
 9   publication_date    11117 non-null  object 
 10  publisher           11117 non-null  object 
 11  language_name       11117 non-null  object 
 12  publisherID         11117 non-null  int64  
dtypes: float64(1), int64(4), object(8)
memory usage: 1.2+ MB


In [14]:
# Step 1: Generate unique authors and their corresponding authorID from books_data
# We will use the unique authors that are already present in the books_data 'author' column
authors_set = set()  # Set to hold unique authors
for author_list in books_data['authors'].dropna():
    authors_set.update(author_list.split('/'))  # Split by '/' if multiple authors

# Step 2: Create authors_df with unique authors and their authorID
authors_df = pd.DataFrame(list(authors_set), columns=['name'])
authors_df['authorID'] = range(1, len(authors_df) + 1)  # Assign unique authorIDs

# Step 3: Save the authors DataFrame to a CSV (optional)
authors_df.to_csv('data/authors.csv', index=False)

# Display authors_df to verify
authors_df.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9190 entries, 0 to 9189
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   name      9190 non-null   object
 1   authorID  9190 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 143.7+ KB


In [15]:
# Step 4: Create a dictionary for fast lookup of authorID by author name
author_to_id = dict(zip(authors_df['name'], authors_df['authorID']))

# Step 5: Update books_data with the correct authorID for each book
# For each book, assign the authorID based on the first author (or any logic you prefer)
books_data['authorID'] = books_data['authors'].apply(
    lambda authors: author_to_id[authors.split('/')[0].strip()] if pd.notna(authors) else None
)

# Display the updated books_data with the new 'authorID' column
books_data.info(), books_data.head()


<class 'pandas.core.frame.DataFrame'>
Index: 11117 entries, 0 to 11118
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   bookID              11117 non-null  int64  
 1   title               11117 non-null  object 
 2   authors             11117 non-null  object 
 3   average_rating      11117 non-null  float64
 4   isbn                11117 non-null  object 
 5   isbn13              11117 non-null  object 
 6     num_pages         11117 non-null  object 
 7   ratings_count       11117 non-null  int64  
 8   text_reviews_count  11117 non-null  int64  
 9   publication_date    11117 non-null  object 
 10  publisher           11117 non-null  object 
 11  language_name       11117 non-null  object 
 12  publisherID         11117 non-null  int64  
 13  authorID            11117 non-null  int64  
dtypes: float64(1), int64(5), object(8)
memory usage: 1.3+ MB


(None,
    bookID                                              title  \
 0       1  Harry Potter and the Half-Blood Prince (Harry ...   
 1       2  Harry Potter and the Order of the Phoenix (Har...   
 2       4  Harry Potter and the Chamber of Secrets (Harry...   
 3       5  Harry Potter and the Prisoner of Azkaban (Harr...   
 4       8  Harry Potter Boxed Set  Books 1-5 (Harry Potte...   
 
                    authors  average_rating        isbn         isbn13  \
 0  JK Rowling/Mary GrandPr            4.57  0439785960  9780439785969   
 1  JK Rowling/Mary GrandPr            4.49  0439358078  9780439358071   
 2               JK Rowling            4.42  0439554896  9780439554893   
 3  JK Rowling/Mary GrandPr            4.56  043965548X  9780439655484   
 4  JK Rowling/Mary GrandPr            4.78  0439682584  9780439682589   
 
     num_pages  ratings_count  text_reviews_count publication_date  \
 0         652        2095690               27591       2006-09-16   
 1         870 

In [16]:
# Step 6: Create book-authors relationships for book_authors.csv
book_authors_data = []

# Iterate through the books_data to generate (bookID, authorID) pairs
for idx, row in books_data.iterrows():
    bookID = row['bookID']  # Assuming 'bookID' column exists in books_data
    authors_list = row['authors']
    
    if pd.notna(authors_list):  
        for author in authors_list.split('/'):  # Split authors if more than one
            author = author.strip()  # Clean up extra spaces
            if author in author_to_id:
                authorID = author_to_id[author]
                book_authors_data.append({'bookID': bookID, 'authorID': authorID})

# Step 7: Create DataFrame
book_authors_df = pd.DataFrame(book_authors_data)

# Remove duplicates where the combination of bookID and authorID is the same
book_authors_df = book_authors_df.drop_duplicates(subset=['bookID', 'authorID'])

# Save book_authors_df to CSV
book_authors_df.to_csv('data/book_authors.csv', index=False)

book_authors_df.head()


,bookID,authorID
0,1,7992
1,1,8358
2,2,7992
3,2,8358
4,4,7992


In [17]:
books_data["bookID"].unique()

array([    1,     2,     4, ..., 45634, 45639, 45641])

In [18]:
books_data.columns

Index(['bookID', 'title', 'authors', 'average_rating', 'isbn', 'isbn13',
       '  num_pages', 'ratings_count', 'text_reviews_count',
       'publication_date', 'publisher', 'language_name', 'publisherID',
       'authorID'],
      dtype='object')

In [19]:
book_authors_df.head(10)

,bookID,authorID
0,1,7992
1,1,8358
2,2,7992
3,2,8358
4,4,7992
5,5,7992
6,5,8358
7,8,7992
8,8,8358
9,9,5811


In [20]:
books_data['text_reviews_count'] = pd.to_numeric(books_data['text_reviews_count'], errors='coerce')

In [21]:
# Step 1: Extract unique publishers from books_data
publishers = set(books_data['publisher'].dropna())  # Remove NaN values and get unique publishers
publishers_df = pd.DataFrame(list(publishers), columns=['name'])

# Step 2: Assign unique publisherID to each publisher
publishers_df['publisherID'] = range(1, len(publishers_df) + 1)

# Step 3: Save the publishers DataFrame to a CSV file
publishers_df = publishers_df[['publisherID', 'name']]  # Ensure columns are in the correct order
publishers_df.to_csv('data/publishers.csv', index=False)
publishers_df.head()


# Step 4: Create a dictionary for fast lookup of publisherID by publisher name
publisher_to_id = dict(zip(publishers_df['name'], publishers_df['publisherID']))

# Step 5: Update books_data with the correct publisherID
books_data['publisherID'] = books_data['publisher'].apply(
    lambda publisher: publisher_to_id[publisher.strip()] if pd.notna(publisher) else None
)

# Display the updated books_data with the new 'publisherID' column
books_data.head()


,bookID,title,authors,average_rating,isbn,isbn13,num_pages,ratings_count,text_reviews_count,publication_date,publisher,language_name,publisherID,authorID
0,1,Harry Potter and the Half-Blood Prince (Harry ...,JK Rowling/Mary GrandPr,4.57,0439785960,9780439785969,652,2095690,27591,2006-09-16,Scholastic Inc.,English,933,7992
1,2,Harry Potter and the Order of the Phoenix (Har...,JK Rowling/Mary GrandPr,4.49,0439358078,9780439358071,870,2153167,29221,2004-09-01,Scholastic Inc.,English,933,7992
2,4,Harry Potter and the Chamber of Secrets (Harry...,JK Rowling,4.42,0439554896,9780439554893,352,6333,244,2003-11-01,Scholastic,English,2268,7992
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,JK Rowling/Mary GrandPr,4.56,043965548X,9780439655484,435,2339585,36325,2004-05-01,Scholastic Inc.,English,933,7992
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,JK Rowling/Mary GrandPr,4.78,0439682584,9780439682589,2690,41428,164,2004-09-13,Scholastic,English,2268,7992


In [22]:
# Step 1: Extract unique languages from the 'language' column in books_data
# Assuming 'language' column exists and contains the language of each book
languages = set(books_data['language_name'].dropna())  # Remove NaN values and get unique languages

# Step 2: Create a DataFrame for languages
languages_df = pd.DataFrame(list(languages), columns=['language_name'])

# Step 3: Save the languages DataFrame to a CSV file
languages_df.to_csv('data/languages.csv', index=False)

# Display the saved file content (optional)
languages_df.head()


,language_name
0,Portuguese
1,Japanese
2,Swedish
3,Serbian
4,Multiple Languages


In [23]:
books_data.columns

Index(['bookID', 'title', 'authors', 'average_rating', 'isbn', 'isbn13',
       '  num_pages', 'ratings_count', 'text_reviews_count',
       'publication_date', 'publisher', 'language_name', 'publisherID',
       'authorID'],
      dtype='object')

In [24]:
books_data.to_csv("data/final_cleaned_data.csv",index=False)

In [25]:
book_authors_df.head(10)

,bookID,authorID
0,1,7992
1,1,8358
2,2,7992
3,2,8358
4,4,7992
5,5,7992
6,5,8358
7,8,7992
8,8,8358
9,9,5811


In [26]:
publishers_df.head(10)

,publisherID,name
0,1,McDougal Littel
1,2,Pavilion Press (Wi)
2,3,Boosey & Hawkes Inc
3,4,Norma
4,5,Grove Press Granta
5,6,Council Press
6,7,Orion Publishing
7,8,Kabel Verlag GmbH Ernst
8,9,Severn House
9,10,McSweeney's


In [27]:
books_data.shape

(11117, 14)